In [1]:
import glob
import random
import pprint
import bayes
import pandas as pd
import time

def import_texts(category, paths):
    data = []
    for path in paths:
        if "LICENSE.txt" in path:
            continue
        
        with open(path, "r") as f:
            s = f.read().split("\n")
            s = "\n".join(s[2:]) # テキストだけ抽出
        
        data.append({
            "label": category,
            "text": s
        })
    return data

peachy = glob.glob("./text/peachy/*.txt")
livedoor = glob.glob("./text/livedoor-homme/*.txt")
sports = glob.glob("./text/sports-watch/*.txt")

dataset = []
dataset += import_texts("peachy", peachy)
dataset += import_texts("livedoor", livedoor)
dataset += import_texts("sports", sports)
random.shuffle(dataset)

test_count = 500
test_data = pd.DataFrame(dataset[:test_count]) 
train_data = pd.DataFrame(dataset[test_count:])

print("Train", len(train_data))
print("Test", len(test_data))

Train 1753
Test 500


In [2]:
%%time

bayes = bayes.NaiveBayes()
bayes.fit("peachy", test_data["text"][test_data["label"] == "peachy"])
bayes.fit("livedoor", test_data["text"][test_data["label"] == "livedoor"])
bayes.fit("sports", test_data["text"][test_data["label"] == "sports"])


#     CPU times: user 141 ms, sys: 3.92 ms, total: 145 ms
#     Wall time: 144 ms


CPU times: user 667 ms, sys: 36 ms, total: 703 ms
Wall time: 710 ms


In [5]:
bayes.classify(test_data["text"][1])

{'peachy': 1.0, 'livedoor': 0.0, 'sports': 0.0}

In [4]:
%%time

ok = 0
ng = 0

for d in test_data.iloc:
    ans = bayes.classify(d["text"])
    
    max_p = max(ans.values())
    label = [k for k, v in ans.items() if v == max_p][0]
    
    if d["label"] == label:
        ok += 1
    else:
        ng += 1
    
#     print(d["text"][:50])
#     print(label, d["label"])
#     print(ans)
#     print()

print("正解率", ok / (ok + ng))


#     CPU times: user 270 ms, sys: 3.94 ms, total: 274 ms
#     Wall time: 273 ms


正解率 0.996
CPU times: user 1.34 s, sys: 445 µs, total: 1.34 s
Wall time: 1.34 s
